In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:

def login(driver):
    driver.get("https://www.facebook.com/")
    email = driver.find_element(By.ID, 'email')
    email.send_keys(os.getenv('EMAIL2'))
    password = driver.find_element(By.ID,'pass')
    password.send_keys('PASWD2')
    driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/form/div[2]/button').click()
def generalNameList():
    x = pd.read_csv('names.csv')
    return x['Generals'].to_list()
def nameToLink(name):
    return name.replace(" ", "%20")
def generalSearch(driver):
    generals = generalNameList()
    master_list = ""
    count = 0
    for general in generals:
        # elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[2]/div[2]/div[1]/div/div/div[3]/div/div/div/div[1]/div/div/label/input')
        # elem.send_keys(general)
        driver.get("https://www.facebook.com/search/people/?q=" + nameToLink(general))
        textual_healing = driver.page_source
        expression = '"profile":{"__typename":"User","__isNode":"User","id":"[0-9]*"'
        ids = re.findall(expression, textual_healing)
        x = np.array(ids)
        p = np.unique(x)
        gen = (general + "/")*len(p)
        p = " ".join(p)
        gen2 = gen + ";" + p
        master_list += str(gen2) + "\n"
        if count%15 == 0 and count != 0:
            time.sleep(20)
        count +=1
    driver.close()
    with open('general_ids.txt', 'w') as f:
        f.write(str(master_list))

# login(driver)
# generalSearch(driver)

In [153]:
def getDataFrame():
    q = open('general_ids.txt', 'r')
    text = q.read()
    text = text.split("\n")
    id_list= []
    for tex in text:
        general_list = tex.split(";")
        general_list[0] = general_list[0].split("/")
        if len(general_list) == 2:
            general_list[1] = general_list[1].split(" ")
            for i in range(len(general_list[1])):
                general_list[1][i] = int(re.sub('\D', '', general_list[1][i]))
            q = list(zip(general_list[0], general_list[1]))
        id_list.append(q)
    actual = []
    for item in id_list:
        actual += item
    hier_index = pd.DataFrame(actual, columns=['General', 'id'])
    df = pd.DataFrame(np.zeros(len(actual)),columns=['score'])
    df = pd.concat([hier_index, df], axis=1)
    df.set_index('id')
    return df


,General,id,score
0,Mark W. Gillette,100021231701450,0.0
1,Mark W. Gillette,100045593106158,0.0
2,Mark W. Gillette,100054796545986,0.0
3,Mark W. Gillette,100062052671178,0.0
4,Mark W. Gillette,100080057053314,0.0
...,...,...,...
656,Tony L. Wright,100006098828370,0.0
657,Tony L. Wright,100009837580460,0.0
658,Tony L. Wright,100075409485562,0.0
659,Tony L. Wright,1676286596,0.0


In [151]:
def getFriendCount(soup):
    return soup.find('span', {"class":"d2edcug0 hpfvmrgz qv66sw1b c1et5uql oi732d6d ik7dh3pa ht8s03o8 a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn iv3no6db a5q79mjw g1cxx5fr b1v8xokw m9osqain"}).get_text()
    # rex = r'"text":".*\sfriends"'
    # friends = re.findall(rex,html)
    # if friends == []:
    #     return 0
    # hell = re.findall(r'(\d+(?:\.\d+)?)', friends[0])
    # if 'K' in friends[0]:
    #     return float(hell[0])*1000
    # else:
    #     return float(hell[0])   
def getName(soup):
    return soup.find('span', {"class":"d2edcug0 hpfvmrgz qv66sw1b c1et5uql oi732d6d ik7dh3pa ht8s03o8 a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 embtmqzv h6olsfn3 mhxlubs3 p5u9llcw hnhda86s oo9gr5id hzawbc8m"}).get_text()
    # name1=soup.title.get_text()
    # name2 = name1.replace('(2)', "")
    # name3 = name2.replace(' | Facebook', '')
    
def getIntro(soup):
    test = soup.find_all('div', class_='rq0escxv l9j0dhe7 du4w35lb j83agx80 cbu4d94t d2edcug0 hpfvmrgz rj1gh0hx buofh1pr g5gj957u o8rfisnq p8fzw8mz pcp91wgn iuny7tx3 ipjc6fyt')
    array = []
    for thing in test:
        if "instagram" not in str(thing) and'twitter' not in str(thing) and 'facebook' not in thing.get_text():
            array.append(thing.get_text())
    array = np.array(array)
    array1 = np.unique(array)
    dict = {"Hometown":"", 'Current_town':"", "Job": []}
    if array1.size == 0:
        return dict
    for item in array1:
        if 'From' in item:
            dict["Hometown"] = item.split("From")[1]
        elif " in " in item and not "Studied" in item:
            dict["Current_town"] = item.split(' in ')[1]
        elif 'Works at ' in item or 'Worked at' in item:
            dict["Job"].append(item.split(' at ')[1])
        elif 'Studied ' not in item and 'Went' not in item and 'From' not in item and 'Lives' not in item and not 'https' in item and ' in ' not in item and "Pronounces" not in item:
            if "It's" not in item and "Widowed" not in item and "Joined" not in item and 'facebook' not in item and 'Divorced' not in item:
                if ' with ' not in item and 'In ' not in item and 'relationship' not in item and 'Married' not in item and "Followed" not in item and "Engaged" not in item and "Single" not in item:
                    dict["Job"].append(item)
    return dict
def getPagecount(soup):
    posts = soup.find_all('div', class_='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0')
    return(len(posts))
def getBannerImageURL(soup):
    bannerImage = soup.find("img", {"data-imgperflogname" : "profileCoverPhoto"})
    if(bannerImage is not None and bannerImage.has_attr('src')):
        return bannerImage['src']
    else:
        return "No Banner"
def getProfilePictureURL(soup):
    q = soup.find('div',class_='b3onmgus e5nlhep0 ph5uu5jm ecm0bbzt spb7xbtv bkmhp75w emlxlaya s45kfl79 cwj9ozl2')
    r = q.find('image')
    rex = r'href=".*" '
    x = re.findall(rex,str(r))
    p = x[0]
    # removed = re.split('xlink:href=")
    almost_there=p.replace('href="', '')
    p = almost_there.replace('"', '')
    image_url = p.replace('amp;', '')
    return image_url
def getPostElements(soup):
    multiple=[]
    posts = soup.find_all('div', class_='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0')
    for post in posts:
        date = post.find('a', class_='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw')
        status = post.find('span', class_='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh jq4qci2q a3bd9o3v b1v8xokw m9osqain')
        image = post.find('img')
        text = post.find('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')
        try:
            link = post.find('a', class_='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 datstx6m k4urcfbm')['href']
        except:
            link = ""
        try:
            text = text.get_text()
        except:
            text = ""
        image=image['src']
        if type(status) == type(None):
            data = {'status':"", 'date':date.get_text()}
        else:
            data = {'status':status.get_text(), 'date': date.get_text(), 'text' : text, 'link':link}
        try:
            url = image.replace('amp;', '')
            data['image_url'] = url
        except:
            data['image_url'] = ""
        multiple.append(data)
    return multiple
def getFriendList(driver):
    driver.get("https://www.facebook.com/profile.php?id="+str(id)+"&sk=friends")
    for i in range(8):
        driver.execute_script("window.scrollTo(0, window.scrollY + 600)")
        time.sleep(1.1)
    html = driver.page_source
    friendlist = []
    soup = bs(html, 'html.parser')
    lister = soup.find('div', class_='j83agx80 btwxx1t3 lhclo0ds i1fnvgqd')
    if not isinstance(lister, type(None)):
        friends = lister.find_all('div', class_='buofh1pr hv4rvrfc')
        for friend in friends:
            name = friend.find('span',class_='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em mdeji52x a5q79mjw g1cxx5fr lrazzd5p oo9gr5id')
            link = friend.find('a')['href']
            name = name.get_text()
            data = {'name':name, 'link':link}
            friendlist.append(data)
    newdata.at[id,'Friend_Info'] = friendlist
    return friendlist
def getPostStats(soup):
    posts = soup.find_all('div', {"class" : "stjgntxs ni8dbmo4 l82x9zwi uo3d90p7 h905i5nu monazrh9"})
    divElements = []
    for post in posts:
        try:
            divElements.append(post.find('div', {"class" : "bp9cbjyn m9osqain j83agx80 jq4qci2q bkfpd7mw a3bd9o3v kvgmc6g5 wkznzc2l oygrvhab dhix69tm jktsbyx5 rz4wbd8a osnr6wyh a8nywdso s1tcr66n"}))
        except:
            divElements.append(None)
    numCommentsList = []
    for commentsElements in divElements:
        try:
            commentElement = commentsElements.find('span',{"class": "d2edcug0 hpfvmrgz qv66sw1b c1et5uql oi732d6d ik7dh3pa ht8s03o8 a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain"})
            if("Comment" in commentElement.get_text()):
                commentText = commentElement.get_text()
            else:
                commentText = "0 Comments"
        except:
            commentText = "0 Comments"
        numCommentsList.append(commentText)
    numSharesList = []
    for shareElements in divElements:
        try:
            shareElement = shareElements.find('span',{"class": "d2edcug0 hpfvmrgz qv66sw1b c1et5uql oi732d6d ik7dh3pa ht8s03o8 a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain"})
            if("Share" in shareElement.get_text()):
                shareText = shareElement.get_text()
            else:
                shareText = "0 Shares"
        except:
            shareText = "0 Shares"
        numSharesList.append(shareText)
    numLikesList = []
    for likeElements in divElements:
        try:
            likeElement = likeElements.find('span',{"class": "gpro0wi8 cwj9ozl2 bzsjyuwj ja2t1vim"})
            likeText = likeElement.get_text() + " Likes"
        except:
            likeText = "0 Likes"
        numLikesList.append(likeText)
    return [numCommentsList, numLikesList, numSharesList]
def nameTest(id):
    global newdata
    general = newdata.at[id, 'General']
    name = newdata.at[id, 'Name']
    name = name.split(' ')
    same = False
    general = general.split(' ')
    if len(general) == len(name) and len(general)==3:
        if general[0].lower() == name[0].lower():
            if general[1][0].lower() == name[1][0].lower():
                if general[2].lower() == name[2].lower():
                    same = True
    elif len(general) == len(name) and len(general)==2:
        if general[0].lower() == name[0].lower():
            if general[1].lower() == name[1].lower():
                same = True
    elif len(general) == 3:
        if general[0].lower() == name[0].lower():
            if general[2].lower() == name[1].lower():
                same = True
    elif len(general)==4 and len(general)==len(name):
        if general[0].lower() == name[0].lower():
            if general[1][0].lower() == name[1][0].lower():
                if general[2].lower() == name[2].lower():
                    if general[3][0:2].lower() == name[3][0:2].lower():
                        same = True

    else:
        if general[0].lower() == name[0].lower():
            if general[1].lower() == name[-1].lower():
                same = True
    if same:
        newdata.at[id,'name_is_same'] = True
    return same
def jobTest(id):
    global newdata
    jobs = newdata.at[id,'Job']
    jobs =jobs[2:len(jobs)-2].split(',')
    if jobs ==[""]:
        newdata.at[id, 'in_army'] = True
        return True
    for job in jobs:
        if"army" in str(job).lower() or 'general' in str(job).lower():
            newdata.at[id, 'in_army'] = True
            return True
    return False
def postTest(id):
    global newdata
    count = newdata.at[id, 'Page_count']
    if count < 5:
        newdata.at[id, 'post_check'] = True
        return True
    else:
        return False
def friendTest(id):
    global newdata
    friends = newdata.at[id, 'Friends']
    if friends < 15:
        newdata.at[id, 'friend_check'] = True
        return True
    else:
        return False
def scoreCalc(idx,checks):
    global newdata
    count =0
    for check in checks:
        if check:
            count+=1
    if count == 1:
        newdata.at[idx, 'score'] = 15
    if count == 2:
        newdata.at[idx, 'score'] = 50
    if count == 3:
        newdata.at[idx, 'score'] = 75
    if count == 4:
        newdata.at[idx, 'score'] = 100





'Thomas L Solhjem'

In [ ]:
driver = webdriver.Firefox(executable_path='./geckodriver.exe')
login(driver)
count = 0
df = getDataFrame()
df.set_index('id', inplace=True)
newdata = pd.DataFrame()
for idx in df.index:
    array = []
    driver.get('https://www.facebook.com/profile.php?id=' + str(idx))
    html = driver.page_source
    friends = findFriends(html)
    info = findInfo(html)
    job = info['Job']
    current_town = info['Current_town']
    home_town = info['Hometown']
    name = findName(html)
    page_count = pseudo_page_count(driver)
    row = pd.DataFrame(data = np.array([[idx,name,job, current_town, home_town, page_count, friends]],dtype=object),columns=['id','Name','Job','Current_town', 'Home_town', 'Page_count', 'Friends'])
    if count%15 == 0 and count != 0:
        time.sleep(3)
    count +=1
    newdata = pd.concat([newdata, row])
driver.close()
newdata.set_index('id', inplace=True)
# datatoexcel = pd.ExcelWriter('facebook.xlsx')
# newdata.to_excel(datatoexcel)
# datatoexcel.save()
newdata


In [ ]:
# driver = webdriver.Firefox(executable_path='./geckodriver.exe')
# login(driver)
# generalSearch(driver)
df = getDataFrame()
data = pd.ExcelFile('facebook.xlsx')
newdata= data.parse('Sheet1')
newdata
newdata = pd.merge(df, newdata, left_on='id', right_on='id')
newdata.set_index('id', inplace=True)
newdata['in_army'] = False
newdata['name_is_same'] = False
newdata['post_check'] = False
newdata['friend_check'] = False
newdata['Banner URL'] = ''
newdata['Post_Metadata'] = ""
newdata['Friend_Info'] = ""
newdata.drop_duplicates(inplace=True)
newdata

In [ ]:
driver = webdriver.Firefox(executable_path='./geckodriver.exe')
login(driver)
for idx in newdata.index:
    driver.get('https://www.facebook.com/profile.php?id=' + str(idx))
    for i in range(8):
        driver.execute_script("window.scrollTo(0, window.scrollY + 600)")
        time.sleep(1.1)
    friendCount = newdata.at[idx, 'Friends']
    getBannerImage(idx, html)
    post_metadata(idx,html)
    profilePicture(idx,html)
    if friendCount>0:
        friendList(idx,driver)
    checks =[jobTest(idx), isNameSame(idx),friendCheck(idx),postCheck(idx)]
    scoreCalc(idx, checks)
newdata

In [146]:
driver = webdriver.Firefox(executable_path='./geckodriver')
login(driver)
driver.get('https://www.facebook.com/profile.php?id=100080332352452')
SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
       print("break")
       break
    last_height = new_height
html = driver.page_source
soup = bs(html, 'html.parser')
getPostStats(html)

    


break


[['1 Comment',
  '0 Comments',
  '4 Comments',
  '0 Comments',
  '0 Comments',
  '0 Comments',
  '0 Comments',
  '1 Comment',
  '2 Comments',
  '3 Comments',
  '17 Comments',
  '0 Comments',
  '0 Comments',
  '1 Comment'],
 ['2 Likes',
  '0 Likes',
  '16 Likes',
  '5 Likes',
  '9 Likes',
  '11 Likes',
  '9 Likes',
  '17 Likes',
  '22 Likes',
  '25 Likes',
  '11 Likes',
  '6 Likes',
  '3 Likes',
  '2 Likes'],
 ['0 Shares',
  '0 Shares',
  '0 Shares',
  '0 Shares',
  '1 Share',
  '0 Shares',
  '0 Shares',
  '0 Shares',
  '0 Shares',
  '0 Shares',
  '0 Shares',
  '3 Shares',
  '0 Shares',
  '0 Shares']]